Option chains
=======

In [24]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)

<IB connected to 127.0.0.1:7496 clientId=12>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [25]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

To avoid issues with market data permissions, we'll use delayed data:

In [26]:
ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [27]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2025, 1, 28, 18, 0, 1, 431223, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=6056.01, lastSize=1.0, high=6060.63, low=5994.63, close=6012.28)

Take the current market value of the ticker:

In [28]:
spxValue = ticker.marketPrice()
spxValue

6056.01

The following request fetches a list of option chains:

In [29]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20250220, 20250320, 20250416, 20250515, 20250...","[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 12..."
1,IBUSOPT,416904,SPX,100,"[20250220, 20250320, 20250416, 20250515, 20250...","[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 12..."
2,CBOE,416904,SPXW,100,"[20250128, 20250129, 20250130, 20250131, 20250...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1200.0, 1..."
3,IBUSOPT,416904,SPXW,100,"[20250128, 20250129, 20250130, 20250131, 20250...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1200.0, 1..."
4,SMART,416904,SPXW,100,"[20250128, 20250129, 20250130, 20250131, 20250...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1200.0, 1..."
5,CBOE,416904,SPX,100,"[20250220, 20250320, 20250416, 20250515, 20250...","[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 12..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [30]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20250220', '20250320', '20250416', '20250515', '20250618', '20250717', '20250814', '20250918', '20251016', '20251120', '20251218', '20260115', '20260219', '20260319', '20260617', '20261217', '20271216', '20281214', '20291220', '20301219'], strikes=[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 1200.0, 1400.0, 1600.0, 1800.0, 1900.0, 2000.0, 2100.0, 2200.0, 2300.0, 2400.0, 2500.0, 2600.0, 2700.0, 2800.0, 2850.0, 2900.0, 2950.0, 3000.0, 3050.0, 3100.0, 3150.0, 3200.0, 3250.0, 3300.0, 3325.0, 3350.0, 3375.0, 3400.0, 3425.0, 3450.0, 3475.0, 3500.0, 3525.0, 3550.0, 3575.0, 3600.0, 3625.0, 3650.0, 3675.0, 3700.0, 3725.0, 3750.0, 3775.0, 3800.0, 3825.0, 3850.0, 3875.0, 3900.0, 3925.0, 3950.0, 3975.0, 4000.0, 4025.0, 4050.0, 4075.0, 4100.0, 4125.0, 4150.0, 4175.0, 4200.0, 4225.0, 4250.0, 4275.0, 4300.0, 4325.0, 4350.0, 4375.0, 4400.0, 4425.0, 4450.0, 4475.0, 4500.0, 4525.0, 4550.0, 4575.0, 

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [31]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

48

In [32]:
contracts[0]

Option(conId=730411519, symbol='SPX', lastTradeDateOrContractMonth='20250220', strike=6040.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   250221P06040000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [38]:
#ask
#bid
tickers = ib.reqTickers(*contracts)



tickers[0]

TypeError: 'Ticker' object is not iterable

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [23]:
ib.disconnect()